In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
def create_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

## 23년 1월 28일 리스트 테스트

In [3]:
date = '20230128'
url = f'https://news.kmib.co.kr/article/list_all.asp?sid1=soc&sid2=&sdate={date}'

soup = create_soup(url)

In [4]:
news_list = soup.find('div', attrs={'class':'nws_list_all brnone'})

In [5]:
news = news_list.find_all('a')

In [6]:
articles = []
base_url = 'https://news.kmib.co.kr/article/'
for i in range(len(news)):
    article = []
    title = news[i].text
    link = base_url + news[i]['href']
    date = news_list.find_all('dd')[i].text
    article = [title, date, link]
    articles.append(article)

## 기사 본문 스크래핑

In [7]:
link = 'https://news.kmib.co.kr/article/view.asp?arcid=0014169099&code=61121611&sid1=soc'
article_soup = create_soup(link)
article_soup = article_soup.find('div', attrs={'id':'articleBody'})
article_soup.div.decompose() # 기사 이미지 설명문 제거

In [8]:
article = article_soup.find_all(text=True)
article = [i for i in article if i != '\n']

In [9]:
article

[' 기사 내용 ',
 '1월 27일 월요일, 아침 날씨입니다.',
 '이 시각 현재 매우 강한 바람이 부는 곳이 있습니다.',
 '아침 최저기온은 서울 3도, 인천 4도, 수원 5도, 춘천 2도, 강릉 3도, 청주 5도, 대전 6도, 전주 6도, 광주 7도, 대구 7도, 부산 9도, 제주 11도로 예상됩니다.',
 '낮 최고기온은 서울 7도, 인천 7도, 수원 8도, 춘천 7도, 강릉 7도, 청주 7도, 대전 7도, 전주 7도, 광주 8도, 대구 8도, 부산 11도, 제주 11도로 예상됩니다.',
 '일부 지방에 강풍특보가 발효중입니다. 시설물 관리에 주의해야 합니다.',
 '이상 날씨였습니다.',
 '웨더봇 기자',
 '※ 이 기사는 국민일보가 개발한 기상뉴스 전용 인공지능 로봇 ‘웨더봇’이 기상청 데이터를 토대로 작성한 것입니다. 지속적인 업그레이드를 통해 더욱 풍부한 내용과 정확한 문장을 담아 가겠습니다.\r\n\t\t\t\t\t',
 '기자페이지 박스',
 '//기자페이지 박스',
 '\r\n\t\t\t\t\tGoodNews paper ⓒ ',
 '국민일보(www.kmib.co.kr)',
 ', 무단전재 및 수집, 재배포금지\r\n\t\t\t\t\t',
 ' ad 20220106 추가 ',
 '// ad 20220106 추가 ']

In [10]:
for i in range(len(article)-1, 0, -1):
    if article[i] == '기자페이지 박스':
        n = i-1
        if article[n][0] == '※':
            n -= 1
        break
content = [i.strip() for i in article[1:n]]
content = ' '.join(content) # 문단 합치기
journalist = article[n].split('기자')[0].strip() + ' 기자'

In [11]:
journalist

'웨더봇 기자'

In [12]:
content, journalist

('1월 27일 월요일, 아침 날씨입니다. 이 시각 현재 매우 강한 바람이 부는 곳이 있습니다. 아침 최저기온은 서울 3도, 인천 4도, 수원 5도, 춘천 2도, 강릉 3도, 청주 5도, 대전 6도, 전주 6도, 광주 7도, 대구 7도, 부산 9도, 제주 11도로 예상됩니다. 낮 최고기온은 서울 7도, 인천 7도, 수원 8도, 춘천 7도, 강릉 7도, 청주 7도, 대전 7도, 전주 7도, 광주 8도, 대구 8도, 부산 11도, 제주 11도로 예상됩니다. 일부 지방에 강풍특보가 발효중입니다. 시설물 관리에 주의해야 합니다. 이상 날씨였습니다.',
 '웨더봇 기자')

## 함수 정리 테스트

In [13]:
# 기사 링크로 본문과 기자 읽어오는 함수
def content_scraper(link):
    article_soup = create_soup(link)
    article_soup = article_soup.find('div', attrs={'id':'articleBody'})
    article_soup.div.decompose() # 기사 이미지 설명문 제거
    article = article_soup.find_all(text=True)
    article = [i for i in article if i != '\n']
    for i in range(len(article)-1, 0, -1):
        if article[i] == '기자페이지 박스':
            n = i-1
            if article[n][0] == '※':
                n -= 1
            break
    content = [i.strip() for i in article[1:n]]
    content = ' '.join(content)
    journalist = article[n].split('기자')[0].strip() + ' 기자'
    return content, journalist

In [14]:
def get_articles(year, month, day):
    year = str(year)
    month = str(month).zfill(2)
    day = str(day).zfill(2)
    date = year+month+day
    url = f'https://news.kmib.co.kr/article/list_all.asp?sid1=soc&sid2=&sdate={date}'
    soup = create_soup(url)

    news_list = soup.find('div', attrs={'class':'nws_list_all brnone'})
    news = news_list.find_all('a')

    articles = []
    base_url = 'https://news.kmib.co.kr/article/'
    # for i in tqdm(range(len(news)), desc=f'{date}'):
    for i in range(len(news)):
        article = []
        title = news[i].text.strip()
        if '[포토]' in title:
            continue
        link = base_url + news[i]['href']
        arcid = link.split('=')[1].split('&')[0]
        date = news_list.find_all('dd')[i].text
        content, journalist = content_scraper(link)
        article = [arcid, title, journalist, date, content, link]
        articles.append(article)
    df = pd.DataFrame(articles, columns=['arcid', 'title', 'journalist', 'date', 'content', 'link'])
    return df

In [15]:
data = get_articles(2023, 1, 9)

## 한달치 스크래핑

In [16]:
from datetime import date
from dateutil.rrule import rrule, DAILY

In [17]:
start_date = date(2022, 12, 29)
end_date = date(2023, 1, 28)

In [18]:
data = pd.DataFrame()
for date in tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
    data = pd.concat([data, get_articles(date.year, date.month, date.day)])
data.reset_index(drop=True, inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [02:27<00:00,  4.76s/it]


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2304 entries, 0 to 2303
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   arcid       2304 non-null   object
 1   title       2304 non-null   object
 2   journalist  2304 non-null   object
 3   date        2304 non-null   object
 4   content     2304 non-null   object
 5   link        2304 non-null   object
dtypes: object(6)
memory usage: 108.1+ KB


In [20]:
data['date'] = pd.to_datetime(data['date'])

In [21]:
data

,arcid,title,journalist,date,content,link
0,0017817579,"축구선수, 배우까지…‘뇌전증 병역 비리 의혹’ 더 커지나",이가현 기자,2022-12-29 23:49:00,허위로 뇌전증 진단을 받아 병역 면제 또는 감면받도록 한 병역 브로커의 구속으로 시...,https://news.kmib.co.kr/article/view.asp?arcid...
1,0017817397,잠옷 차림으로 9층 베란다 넘으려다 체포…막 내린 도주극,신지호 기자,2022-12-29 20:27:00,전자팔찌를 끊고 도주했던 ‘라임자산운용 사태’ 핵심 인물 김봉현 전 스타모빌리티 회...,https://news.kmib.co.kr/article/view.asp?arcid...
2,0017817389,제2경인고속도 화재 사고… ‘플라스틱 벽’이 피해 키웠나,이가현 기자,2022-12-29 20:22:00,경기 과천시 제2경인고속도로 방음 터널에서 29일 발생한 화재 사고 피해가 커진 원...,https://news.kmib.co.kr/article/view.asp?arcid...
3,0017817056,"“방수복·오리발이 그대로”…檢, ‘월북’ 아닌 ‘실족’ 무게",조민아 신지호 기자,2022-12-29 19:58:00,서해 공무원 고(故) 이대준씨 ‘월북 몰이’ 사건 당시 첩보 삭제를 지시한 혐의로 ...,https://news.kmib.co.kr/article/view.asp?arcid...
4,0017817334,‘갑질 논란’ 신창현 수도권매립지공사 사장 해임,인천=김민 기자,2022-12-29 19:37:00,신창현(69) 수도권매립지관리공사 사장이 취임 1년 5개월 만에 결국 해임됐다. 2...,https://news.kmib.co.kr/article/view.asp?arcid...
...,...,...,...,...,...,...
2299,0924284597,30일부터 ‘노마스크’ 수업 가능… 통학버스 이용 땐 벗으면 안돼,성윤수 기자,2023-01-28 04:03:00,30일 실내 마스크 착용 의무 조정을 앞두고 일부 혼선 조짐이 나타나고 있다. 시민...,https://news.kmib.co.kr/article/view.asp?arcid...
2300,0924284593,"‘해직교사 특별채용’ 의혹 조희연 서울시교육감, 징역 1년6월·집행유예 2년",신지호 기자,2023-01-28 04:03:00,해직 교사를 부당하게 특별채용한 혐의로 기소된 조희연( 사진 ) 서울시교육감이 1심...,https://news.kmib.co.kr/article/view.asp?arcid...
2301,0924284596,60세 정년은 옛말?… 고령층 ‘계속 고용’ 논의 본격 착수,조효석 기자,2023-01-28 04:01:00,정부가 직장 내 정년연장을 위한 사회적 논의에 착수하겠다고 발표했다. 고용노동부는 ...,https://news.kmib.co.kr/article/view.asp?arcid...
2302,0924284594,국민연금 2055년 고갈… 고강도 개혁 속도낼 듯,조효석 기자,2023-01-28 04:00:00,국민연금 재정소진 예상연도가 5년 전 전망치보다 2년 더 당겨진 것으로 나타났다. ...,https://news.kmib.co.kr/article/view.asp?arcid...


## TfidfVectorizer

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
with open('.\data\stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [line.rstrip('\n') for line in stopwords]

In [24]:
ftidf_vec = TfidfVectorizer(stop_words=stopwords)
tfidf_dtm = ftidf_vec.fit_transform(data['content'])

C:\Users\user\anaconda3\envs\sesac\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(


In [25]:
cos_sim_res = cosine_similarity(tfidf_dtm, tfidf_dtm)

In [26]:
arcid_to_index = dict(zip(data['arcid'], data.index))

In [27]:
def get_recommendation(title, n):
    arcid = data.loc[data['title'] == title]['arcid'].values[0]
    idx = arcid_to_index[arcid]
    sim_scores = list(enumerate(cos_sim_res[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    return data.iloc[article_idx][['title', 'date', 'link']]

In [29]:
get_recommendation('잠옷 차림으로 9층 베란다 넘으려다 체포…막 내린 도주극', 5)

,title,date,link
120,‘라임 주범’ 김봉현 9층 베란다 넘으려다 체포,2022-12-30 04:08:00,https://news.kmib.co.kr/article/view.asp?arcid...
1369,검찰 ‘라임 사태 핵심’ 김봉현에 징역 40년 구형,2023-01-16 18:21:00,https://news.kmib.co.kr/article/view.asp?arcid...
16,‘라임 사태 핵심’ 김봉현 화성에서 검거…도주 48일만,2022-12-29 16:46:00,https://news.kmib.co.kr/article/view.asp?arcid...
1144,"‘라임 몸통’ 김봉현, 도주극 뒤 첫 재판 ‘또’ 불출석",2023-01-12 14:49:00,https://news.kmib.co.kr/article/view.asp?arcid...
1558,‘라임’ 김봉현 징역 40년 구형… 檢 “도주 순간 중범죄자 자인”,2023-01-17 04:07:00,https://news.kmib.co.kr/article/view.asp?arcid...


In [ ]:
# TO-DO
# Okt적용, 외부기사 링크로 스크래핑해서 내용을 transform->유사도 연산->추천